In [1]:
import pandas as pd
from sqlalchemy import create_engine

import sys, os

# Helpers
abspath = os.path.abspath
dirname = os.path.dirname
sep = os.sep
file_ = os.getcwd()

sys.path.append(dirname(abspath(file_)))

import utils.folder_tb as fo
import utils.mining_data_tb as md
import utils.sql_tb as sq

## Data to upload

In [4]:
# Load the data and check that everything is fine
cleaned_data_path = fo.path_to_folder(2, "data" + sep + "7_cleaned_data") + "cleaned_data.csv"
data = pd.read_csv(cleaned_data_path)

print(data.shape)
data.info()

(13472, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13472 entries, 0 to 13471
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      13472 non-null  int64  
 1   MCQ010    13472 non-null  float64
 2   RIAGENDR  13472 non-null  int64  
 3   RIDAGEYR  13472 non-null  int64  
 4   DR1TCHOL  13472 non-null  float64
 5   DR1TTFAT  13472 non-null  float64
 6   DR1TSFAT  13472 non-null  float64
 7   DR1TSUGR  13472 non-null  float64
 8   DR2TCHOL  13472 non-null  float64
 9   DR2TTFAT  13472 non-null  float64
 10  DR2TSFAT  13472 non-null  float64
 11  DR2TSUGR  13472 non-null  float64
 12  BPXDI1    13472 non-null  float64
 13  BPXSY1    13472 non-null  float64
 14  BMXWT     13472 non-null  float64
 15  BMXWAIST  13472 non-null  float64
 16  LBXTC     13472 non-null  float64
 17  LBXSGL    13472 non-null  float64
dtypes: float64(15), int64(3)
memory usage: 1.9 MB


In [7]:
model_comparisons_path = fo.path_to_folder(2, "models")

# First model table
model_comparison_noscale_nobalance = pd.read_csv(model_comparisons_path + "model_comparison_noscale_nobalance.csv")
model_comparison_noscale_nobalance.rename(columns = {"Unnamed: 0" : "Model"}, inplace = True)

# Second model table
model_comparison_scale_balance = pd.read_csv(model_comparisons_path + "model_comparison_scale_balance.csv")
model_comparison_scale_balance.rename(columns = {"Unnamed: 0" : "Model"}, inplace = True)

In [12]:
model_comparison_noscale_nobalance

,Model,Test_score,Train_score,Test_score_drop,Accuracy,Precision,Recall,F1_score,Confusion_matrix
0,"LogisticRegression(max_iter=300, n_jobs=-1, ra...",0.922305,0.915675,0.007240,0.922305,0.617021,0.131818,0.217228,[[2452 18]\n [ 191 29]]
1,"LogisticRegression(n_jobs=-1, random_state=42)",0.920818,0.916233,0.005004,0.920818,0.574468,0.122727,0.202247,[[2450 20]\n [ 193 27]]
2,"RandomForestClassifier(max_depth=10, n_jobs=-1...",0.919703,0.936036,-0.017449,0.919703,0.750000,0.027273,0.052632,[[2468 2]\n [ 214 6]]
3,"RandomForestClassifier(max_depth=20, n_jobs=-1...",0.919703,0.971458,-0.053276,0.919703,0.571429,0.072727,0.129032,[[2458 12]\n [ 204 16]]
4,"RandomForestClassifier(n_jobs=-1, random_state...",0.918587,0.971830,-0.054786,0.918587,0.526316,0.045455,0.083682,[[2461 9]\n [ 210 10]]
5,"RandomForestClassifier(max_features='sqrt', n_...",0.918587,0.971830,-0.054786,0.918587,0.526316,0.045455,0.083682,[[2461 9]\n [ 210 10]]
6,"RandomForestClassifier(max_features='log2', n_...",0.918587,0.971830,-0.054786,0.918587,0.526316,0.045455,0.083682,[[2461 9]\n [ 210 10]]
7,KNeighborsClassifier(n_jobs=-1),0.914126,0.917627,-0.003815,0.914126,0.342857,0.054545,0.094118,[[2447 23]\n [ 208 12]]
8,"KNeighborsClassifier(n_jobs=-1, weights='dista...",0.914126,0.970342,-0.057934,0.914126,0.342857,0.054545,0.094118,[[2447 23]\n [ 208 12]]


## SQL connection

In [2]:
# Load server setting into variables
read_json = md.read_json_to_dict("sql_server_settings.json")

IP_DNS = read_json["IP_DNS"]
USER = read_json["USER"]
PASSWORD = read_json["PASSWORD"]
DB_NAME = read_json["DB_NAME"]
PORT = read_json["PORT"]

In [3]:
# Connection with the database
sql_db = sq.MySQL(IP_DNS, USER, PASSWORD, DB_NAME, PORT)
sql_db.connect()

Connected to MySQL server [45395203b_ds_april_2021_db]


In [11]:
db_connection_str = sql_db.SQL_ALCHEMY
db_connection = create_engine(db_connection_str)

In [ ]:
### DON'T EXECUTE THIS ONE

# Uploading the data into the db
data.to_sql("jonathan_suarez_caceres", con = db_connection, index = False)

In [13]:
# First table
model_comparison_scale_balance.to_sql("model_comparison_scale_balance", con = db_connection, index = False)

# Second table
model_comparison_noscale_nobalance.to_sql("model_comparison_noscale_nobalance", con = db_connection, index = False)

In [14]:
# Close connection
sql_db.close()

Close connection with MySQL server [45395203b_ds_april_2021_db]


In [10]:
sql_query = '''
SELECT * FROM model_comparison_noscale_nobalance
'''

results = sql_db.execute_get_sql(sql_query)
column_names = [tuple[0] for tuple in sql_db.cursor.description]
pd.DataFrame(results, columns = column_names)

Executing:
 
SELECT * FROM model_comparison_noscale_nobalance



,Model,Test_score,Train_score,Test_score_drop,Accuracy,Precision,Recall,F1_score,Confusion_matrix
0,"LogisticRegression(max_iter=300, n_jobs=-1, ra...",0.922305,0.915675,0.007240,0.922305,0.617021,0.131818,0.217228,[[2452 18]\n [ 191 29]]
1,"LogisticRegression(n_jobs=-1, random_state=42)",0.920818,0.916233,0.005004,0.920818,0.574468,0.122727,0.202247,[[2450 20]\n [ 193 27]]
2,"RandomForestClassifier(max_depth=10, n_jobs=-1...",0.919703,0.936036,-0.017449,0.919703,0.750000,0.027273,0.052632,[[2468 2]\n [ 214 6]]
3,"RandomForestClassifier(max_depth=20, n_jobs=-1...",0.919703,0.971458,-0.053276,0.919703,0.571429,0.072727,0.129032,[[2458 12]\n [ 204 16]]
4,"RandomForestClassifier(n_jobs=-1, random_state...",0.918587,0.971830,-0.054786,0.918587,0.526316,0.045455,0.083682,[[2461 9]\n [ 210 10]]
5,"RandomForestClassifier(max_features='sqrt', n_...",0.918587,0.971830,-0.054786,0.918587,0.526316,0.045455,0.083682,[[2461 9]\n [ 210 10]]
6,"RandomForestClassifier(max_features='log2', n_...",0.918587,0.971830,-0.054786,0.918587,0.526316,0.045455,0.083682,[[2461 9]\n [ 210 10]]
7,KNeighborsClassifier(n_jobs=-1),0.914126,0.917627,-0.003815,0.914126,0.342857,0.054545,0.094118,[[2447 23]\n [ 208 12]]
8,"KNeighborsClassifier(n_jobs=-1, weights='dista...",0.914126,0.970342,-0.057934,0.914126,0.342857,0.054545,0.094118,[[2447 23]\n [ 208 12]]


In [8]:
[tuple[0] for tuple in sql_db.cursor.description]

['Model',
 'Test_score',
 'Train_score',
 'Test_score_drop',
 'Accuracy',
 'Precision',
 'Recall',
 'F1_score',
 'Confusion_matrix']